In [1]:
from pyspark import SparkContext
from pyspark.sql.functions import countDistinct
sc=SparkContext()

22/11/25 18:47:34 WARN Utils: Your hostname, Tobiass-MacBook.local resolves to a loopback address: 127.0.0.1; using 10.27.90.105 instead (on interface en0)
22/11/25 18:47:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/25 18:47:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/25 18:47:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [26]:
import numpy as np


Pseude Code:

agrupar por user, device, gt
    para x, y, z sacar
        median
        desv
        max
        min

join data from different datasets into RDDs
Analize RDDs

Don´t repeat code!
avoid many (unnecessary) lines of code / repetitions

In [20]:
#columns = ["index", "arrival_time", "creation_time", "x", "y", "z", "user", "model", "device", "activity"]

pa = sc.textFile('Phones_accelerometer.csv')

pg = sc.textFile('Phones_gyroscope.csv')

wa = sc.textFile('Watch_accelerometer.csv')

wg = sc.textFile('Watch_gyroscope.csv')


In [123]:
split = wg.map(lambda x : x.split(","))
keyed = split.map(lambda x : (x[6] + "," + x[7]+ "," + x[9], np.array([1, float(x[3]), float(x[4]), float(x[5]), float(x[3])**2, float(x[4])**2, float(x[5])**2])))

sum = keyed.reduceByKey(lambda x, y : x + y)
std = sum.map(lambda x : (x[0], np.array([np.sqrt((x[1][4]/x[1][0])-(x[1][1]/x[1][0])**2), np.sqrt((x[1][5]/x[1][0])-(x[1][2]/x[1][0])**2), np.sqrt((x[1][6]/x[1][0])-(x[1][3]/x[1][0])**2)])))
mean = sum.map(lambda x : (x[0], np.array((x[1][1]/x[1][0], x[1][2]/x[1][0], x[1][3]/x[1][0]))))

#sum_std = keyed.map(lambda x: (x[1][1]  - mean[1][1])**2)
'''std = sum_std.reduce(lambda x,y: x + y)
std = np.sqrt(std/heights.count())'''

sum.collect()

[('a,gear,stand',
  array([5047.        ,  111.77420486, -165.88138325, -360.541152  ,
           29.48112279,   21.11171523,   38.46312038])),
 ('a,gear,null',
  array([ 3.43000000e+02,  7.74154314e+00, -1.25568048e+01, -2.45913637e+01,
          2.83401834e-01,  5.05536017e-01,  1.78524658e+00])),
 ('a,gear,sit',
  array([4610.        ,  104.67288548, -190.95770893, -322.57645873,
           17.37226198,   22.50625132,   32.00580686]))]

In [132]:
def read_in(filename):
    #read in file and split at ","
    rdd = sc.textFile(filename)
    split_rdd = rdd.map(lambda x : x.split(","))

    #remove unnecessary variables, create primary key (user + model + gt), create floats: counter variable, x, y, z
    keyed = split_rdd.map(lambda x : (x[6] + "," + x[7]+ "," + x[9], np.array([1, float(x[3]), float(x[4]), float(x[5])])))
    return keyed

In [141]:
def calc_mean(rdd):
    # sum up numericals from input rdd --> x[1]: count per key, sum(x), sum(y), sum(z)
    sum = rdd.reduceByKey(lambda x, y : x + y)
    # divide sums by count
    mean = sum.map(lambda x : (x[0], np.array((x[1][1]/x[1][0], x[1][2]/x[1][0], x[1][3]/x[1][0]))))

    return mean


In [134]:
def calc_std(rdd):
    #remove unnecessary variables, create primary key (user + model + gt), create floats: counter variable, x, y, z, x^2, y^2, z^2 
    work_rdd = rdd.map(lambda x : (x[0], np.array([1, x[1][1], x[1][2], x[1][3], (x[1][1])**2, (x[1][2])**2, (x[1][3])**2])))

    # sum up individual numerical values
    sum = work_rdd.reduceByKey(lambda x, y : x + y)
    # calculate std as follows: sqrt( (sum(x^2)/N) - (sum(x)/N)^2 )
    std = sum.map(lambda x : (x[0], np.array([np.sqrt((x[1][4]/x[1][0])-(x[1][1]/x[1][0])**2), np.sqrt((x[1][5]/x[1][0])-(x[1][2]/x[1][0])**2), np.sqrt((x[1][6]/x[1][0])-(x[1][3]/x[1][0])**2)])))

    return std

In [411]:
def find_max(rdd):
    rdd = read_in('Watch_gyroscope.csv')
    rdd = rdd.filter(lambda x : x[0] == 'a,gear,null')
    first_line = rdd.take(1)
    count = rdd.count()
    print(rdd.count())

    while count > 0:
        max_x = first_line[0][1][1]
        maxed_x = rdd.filter(lambda x :  x[1][1] > max_x )
        count = maxed_x.count()
        first_line = maxed_x.take(1)
    
    count = rdd.count()
    first_line = rdd.take(1)
    while count > 0:
        max_y = first_line[0][1][2]
        maxed_y = rdd.filter(lambda x :  x[1][2] > max_y )
        count = maxed_y.count()
        first_line = maxed_y.take(1)
    
    count = rdd.count()
    first_line = rdd.take(1)
    while count > 0:
        max_z = first_line[0][1][3]
        maxed_z = rdd.filter(lambda x :  x[1][3] > max_z )
        count = maxed_z.count()
        first_line = maxed_z.take(1)

    return [max_x, max_y, max_z]

In [413]:
rdd = read_in('Watch_gyroscope.csv')
#out = rdd.filter(lambda x : x[0] == 'a,gear,null')
out = find_max(rdd)
out

343


[0.0561927, -0.018109497, -0.055127434]

In [483]:
def get_keys(rdd):
    
    out = rdd.reduceByKey(lambda x, y: x+y)


    collected = out.collect()
    keys = []
    for i in range(out.count()):
        keys.append(collected[i][0])

    return keys


In [484]:
def find_max(rdd):
    rdd = read_in('Watch_gyroscope.csv')
    keys = get_keys(rdd)
    out = sc.emptyRDD()
    #print(out.collect(), "break\n")

    for i in range(len(keys)):
        key = keys[i]
        #print(key)
        max_values = np.empty([3,1])
        for i in range(3):
            current = rdd.filter(lambda x : x[0]==key)
            #print(current.take(1))
            current_x = current.map(lambda x: (x[0], x[1][i+1]))
            max_x = current_x.max()
            max_values[i] = max_x[1]
            #print(max_x[1])
        
        entry = (str(key), max_values)
        entry_rdd = sc.parallelize(entry)
        #print(entry_rdd.collect())
        out = out.union(entry_rdd)
    #out = out.map(lambda x : (x[0], list(x[1:])))
    #out.collect()
    return out

In [482]:
rdd = read_in('Watch_gyroscope.csv')
keys = get_keys(rdd)
out = sc.emptyRDD()
#print(out.collect(), "break\n")

for i in range(len(keys)):
    key = keys[i]
    #print(key)
    max_values = np.empty([3,1])
    for i in range(3):
        current = rdd.filter(lambda x : x[0]==key)
        #print(current.take(1))
        current_x = current.map(lambda x: (x[0], x[1][i+1]))
        max_x = current_x.max()
        max_values[i] = max_x[1]
        #print(max_x[1])
    
    entry = (str(key), max_values)
    entry_rdd = sc.parallelize(entry)
    #print(entry_rdd.collect())
    out = out.union(entry_rdd)
#out = out.map(lambda x : (x[0], list(x[1:])))
out.collect()

['a,gear,stand',
 array([[0.81039995],
        [0.35446674],
        [1.1475562 ]]),
 'a,gear,null',
 array([[ 0.0561927 ],
        [-0.0181095 ],
        [-0.05512743]]),
 'a,gear,sit',
 array([[0.39867523],
        [0.39920786],
        [0.77604514]])]

In [507]:
def find_min(rdd):
    rdd = read_in('Watch_gyroscope.csv')
    keys = get_keys(rdd)
    out = sc.emptyRDD()
    #print(out.collect(), "break\n")

    for i in range(len(keys)):
        key = keys[i]
        #print(key)
        max_values = np.empty([3,1])
        for i in range(3):
            current = rdd.filter(lambda x : x[0]==key)
            #print(current.take(1))
            current_x = current.map(lambda x: (x[0], x[1][i+1]))
            max_x = current_x.min()
            max_values[i] = max_x[1]
            #print(max_x[1])
        
        entry = (str(key), max_values)
        entry_rdd = sc.parallelize(entry)
        #print(entry_rdd.collect())
        out = out.union(entry_rdd)
    #out = out.map(lambda x : (x[0], list(x[1:])))
    #out.collect()
    return out

In [508]:
def process(filename):
    rdd = read_in(filename)
    mean = calc_mean(rdd)
    std = calc_std(rdd)
    max = find_max(rdd)
    min = find_min(rdd)

    out = mean.union(std)
    out = out.union(max)
    out = out.union(min)
    return out

In [509]:
out = process('Watch_gyroscope.csv')
out.collect()

[('a,gear,stand', array([ 0.02214666, -0.03286732, -0.07143673])),
 ('a,gear,null', array([ 0.0225701 , -0.03660876, -0.07169494])),
 ('a,gear,sit', array([ 0.02270562, -0.0414225 , -0.0699732 ])),
 ('a,gear,stand', array([0.07314945, 0.05570244, 0.0501775 ])),
 ('a,gear,null', array([0.01779986, 0.01156134, 0.00803968])),
 ('a,gear,sit', array([0.05703369, 0.05626924, 0.04523762])),
 'a,gear,stand',
 array([[0.81039995],
        [0.35446674],
        [1.1475562 ]]),
 'a,gear,null',
 array([[ 0.0561927 ],
        [-0.0181095 ],
        [-0.05512743]]),
 'a,gear,sit',
 array([[0.39867523],
        [0.39920786],
        [0.77604514]]),
 'a,gear,stand',
 array([[-2.0383835 ],
        [-0.55287224],
        [-1.2319783 ]]),
 'a,gear,null',
 array([[-0.0314253 ],
        [-0.08042747],
        [-0.09347696]]),
 'a,gear,sit',
 array([[-0.700944  ],
        [-0.79042625],
        [-0.22849922]])]